# Comparing nonlinearities

Running ReLU, LReLU, Sigmoid, Swish, and SQGL (with and without synapses)

In [2]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from fit_params.fit_functions import *

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter

from custompackage.load_architecture import *
from custompackage.load_data import *
from custompackage.traintestloop import *

import sparselinear as sl
import custompackage.sl_custom as slc

import hyperopt as hp
%matplotlib inline

np.set_printoptions(suppress=True)

import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

## Initialize class for running each nonlinearity for all datasets

In [18]:
class Nonlinearities():
    '''
    Establish nonlinearity, synapse status, directory, leak
    
    '''
    def __init__(self, model_type='ktree', activation='relu', selector=0, dataset='mnist', 
                 DIR='./results/temp/', Synapse=False, leak=0.01):
        
        self.model_type = model_type
        self.activation = activation
        self.selector = selector
        self.dataset = dataset
        self.DIR = DIR
        self.Synapse = Synapse
        self.leak = leak
        
        if self.Synapse == True:
            self.syn_key = 'syn'
        else:
            self.syn_key = 'nosyn'
        
    def objective(self):
        # Learning rate
        
        lr_dict = pickle.load(open('./results/hyperparameters/'+self.model_type+'_lr.hyperopt', 'rb'))
            
        lr = lr_dict[self.activation][self.dataset][self.syn_key]['lr']
        
        # SQGL parameters if appropriate
        atten = 0.5
        scale = 1
        
        # Initialize settings
        bs = 256
        weighting = 'paired'
        trials = 10
        epochs = 2000
        trees_set = [1, 2, 4, 8, 16, 32]

        # Load class-dataset list
        classes = np.load('./results/classes.npy', allow_pickle=True)
        classes = [classes[self.selector]]

        # Initialize recording
        acc = np.zeros((len(classes), trials, len(trees_set)))
        loss = np.zeros((len(classes), trials, len(trees_set)))
        
        # For each dataset enumerated from classes list
        for j, (t1, t2, ds) in enumerate(classes):
            print(t1, t2, ds)
            # Load data loaders
            trainloaders, validloaders, testloader = dataset_weighted_split_all(bs, t1, t2, weighting, trials, ds, permute=False)
            # Initialize input size for model initialization purposes
            input_size = trainloaders[0].dataset.tensors[0][0].shape[0]
            # For each trial
            for i in range(trials):
                # For each tree number:
                for k, trees in enumerate(trees_set):
                    print(j, i, k)
                    # Initialize the model
                    if self.model_type == 'fcnn':
                        if self.Synapse == True:
                            model = synapse_fcnn(Input_size=input_size, Hidden_size=trees*2, Output_size=1,
                                                 Activation=self.activation, learn=False, alpha=1, beta=1, gamma=1, 
                                                 scale=scale, atten=atten, leak=self.leak).cuda()
                        else:
                            model = simple_fcnn(Input_size=input_size, Hidden_size=trees*2, Output_size=1,
                                                 Activation=self.activation, learn=False, alpha=1, beta=1, gamma=1, 
                                                 scale=scale, atten=atten, leak=self.leak).cuda()
                    else:
                        model = ktree_sparse(ds=ds, Repeats=trees, Padded=True, Activation=self.activation, 
                                             learn=False, alpha=1, beta=1, gamma=1, scale=scale, atten=atten,
                                             synapse=self.Synapse).cuda()
                    #Train and test ktree, assigning loss and acc values
                    loss_curve, acc_curve, loss[j,i,k], acc[j,i,k], model_t = train_test_ktree_sparse(model, trainloaders[i],
                                                                                        validloaders[i], testloader, 
                                                                                        epochs = epochs, randorder=False,
                                                                                        lr=lr)
                    
                    np.save(self.DIR+'acc_'+'_'.join((self.model_type,self.activation,self.syn_key,str(self.leak*100),self.dataset)), acc)
                    np.save(self.DIR+'loss_'+'_'.join((self.model_type,self.activation,self.syn_key,str(self.leak*100),self.dataset)), loss)

## Run nonlinearities for all classes - ktree

### ReLU

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='ktree', activation='relu', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=False, leak=0)
    testing_loop.objective()

### LReLU

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='ktree', activation='relu', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=False, leak=0.01)
    testing_loop.objective()

### Sigmoid

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='ktree', activation='sigmoid', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=False, leak=0)
    testing_loop.objective()

### Swish

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='ktree', activation='swish', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=False, leak=0)
    testing_loop.objective()

### SQGL nosyn

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='ktree', activation='SQGL', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=False, leak=0)
    testing_loop.objective()

### SQGL syn

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='ktree', activation='SQGL', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=True, leak=0)
    testing_loop.objective()

## Run nonlinearities for all classes - FCNN

### ReLU

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='fcnn', activation='relu', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=False, leak=0)
    testing_loop.objective()

### LReLU

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='fcnn', activation='relu', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=False, leak=0.01)
    testing_loop.objective()

### Sigmoid

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='fcnn', activation='sigmoid', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=False, leak=0)
    testing_loop.objective()

### Swish

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='fcnn', activation='swish', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=False, leak=0)
    testing_loop.objective()

### SQGL nosyn

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='fcnn', activation='SQGL', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=False, leak=0)
    testing_loop.objective()

### SQGL syn

In [ ]:
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    testing_loop = Nonlinearities(model_type='fcnn', activation='SQGL', selector=selector, dataset=dataset, 
                 DIR='./results/20210102/', Synapse=True, leak=0)
    testing_loop.objective()